# Workflow for modeling

Using outputs from data_sprint_anna. Duplicating the same logic for folder names etc.

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
import scipy.special
import numpy as np
import pandas as pd

In [2]:
sampling = 1 # 0 for random sampling, 1 for Static Sparce Grid
num_samples = 15 # if using SSG, it will find the level to get at least this amount of samples

#set the bounds, order t_eped, n_eped, d_n_ped (=d_T_ped), n_esep
bounds = [[1.0, 1.8], [3.0, 4.4], [0.05, 0.1], [0.5, 2.8]]

#set output directory (can be empty or non-empty for analysis)
if sampling == 0:
    base_run_dir = '/scratch/project_462000451/daniel/sprint_out/helena/full_test'
else:
    base_run_dir = '/scratch/project_462000451/daniel/sprint_out/helena/19_2_SSG'

In [3]:

run_folder_name = '-'.join([str(b) for b in np.array(bounds).flatten()])
hel_output_names = []
if os.path.exists(os.path.join(base_run_dir, run_folder_name)):
    hel_output_names = [name for name in os.listdir(os.path.join(base_run_dir, run_folder_name)) 
                    if os.path.isdir(os.path.join(base_run_dir, run_folder_name, name))]
    print("The following HELENA output directories were found in the folder")
    print(hel_output_names)
else:
    print(os.path.join(base_run_dir, run_folder_name))
    print("Given directory does not excist")

The following HELENA output directories were found in the folder
['1.4-3.7-0.07500000000000001-0.07500000000000001-0.5000000000000003', '1.4-4.4-0.07500000000000001-0.07500000000000001-1.65', '1.8-3.7-0.07500000000000001-0.07500000000000001-2.8', '1.8-3.7-0.07500000000000001-0.07500000000000001-1.65', '1.4-3.7-0.07500000000000001-0.07500000000000001-2.8', '1.4-3.7-0.1-0.1-2.8', '1.4-3.7-0.1-0.1-1.65', '1.8-4.4-0.07500000000000001-0.07500000000000001-1.65', '1.4-3.7-0.05000000000000001-0.05000000000000001-1.65', '1.4-3.7-0.07500000000000001-0.07500000000000001-1.65', '1.4-4.4-0.1-0.1-1.65', '1.0-3.7-0.07500000000000001-0.07500000000000001-1.65', '1.4-4.4-0.07500000000000001-0.07500000000000001-2.8', '1.4-3.0-0.07500000000000001-0.07500000000000001-1.65', '1.8-3.7-0.1-0.1-1.65']


In [4]:
sys.path.append('/project/project_462000451/enchanted-surrogates_11feb2025/submodules')
sys.path.append('/project/project_462000451/enchanted-surrogates_11feb2025/submodules/IFS_scripts')
sys.path.append('/project/project_462000451/enchanted-surrogates_11feb2025/src')
from parsers.GENEparser import GENEparser
gp = GENEparser()
scanfiles_dir = '/scratch/project_462000451/gene_out/gene_auto_97781/jet_97781_ion-scale_90/ex-jet_97781_ion-scale_90_batch-0/scanfiles0000/'
print(sys.path)
x = gp.get_model_inputs(scanfiles_dir, suffix='0001')
y = gp.get_model_outputs(scanfiles_dir, suffix='0001')

print('x',x)
print('y',y)


Cannot find DREAM. Please add the $DREAMPATH/py to your PYTHONPATH environment variable before running. No module named 'DREAM'
['/opt/cray/pe/python/3.11.7/lib/python311.zip', '/opt/cray/pe/python/3.11.7/lib/python3.11', '/opt/cray/pe/python/3.11.7/lib/python3.11/lib-dynload', '', '/scratch/project_462000451/daniel/daniel_sprint/lib/python3.11/site-packages', '/project/project_462000451/enchanted-surrogates_11feb2025/submodules', '/project/project_462000451/enchanted-surrogates_11feb2025/submodules/IFS_scripts', '/project/project_462000451/enchanted-surrogates_11feb2025/src']
GETTING MODEL INPUTS
NORM GRAD [3.5749413, 0.48783932, 0.48783932]
GEOM COEFF {'ggxx': array([1.58952518, 1.36367522, 1.07569167, 0.79868977, 0.5732253 ,
       0.40530722, 0.29540943, 0.23627651, 0.23276749, 0.30311948,
       0.46990853, 0.76918592, 1.20831272, 1.80906419, 2.56913871,
       3.6384146 , 4.85035111, 5.97834803, 6.36365   , 5.81165154,
       4.53793567, 3.22842117, 2.25843186, 1.63386614, 1.2391

# Get inputs and outputs from GENE runs

In [5]:
gene_base_dir = '/scratch/project_462000451/daniel/sprint_out/gene/19_2_SSG'
#outputs = gp.get_model_outputs('/scratch/project_462000451/daniel/sprint_out/gene/full_test/1.0-1.8-3.0-4.4-0.05-0.1-0.5-2.8/1.6907219221235361-4.066386570204081-0.08957719078481746-0.08957719078481746-1.6580178585501517/scanfiles0001', '0001')
#inputs = gp.get_model_inputs('/scratch/project_462000451/daniel/sprint_out/gene/full_test/1.0-1.8-3.0-4.4-0.05-0.1-0.5-2.8/1.6907219221235361-4.066386570204081-0.08957719078481746-0.08957719078481746-1.6580178585501517/scanfiles0001', '0001')
#print(inputs)
#print(outputs)

outputs = []
inputs = []

for name_path in hel_output_names:
    scan_sufxs = gp.get_all_suffixes(os.path.join(gene_base_dir, run_folder_name, name_path))
    last_scan_sufx = scan_sufxs[-1]
    scan_file_path = os.path.join(gene_base_dir, run_folder_name, name_path, "scanfiles"+last_scan_sufx)
    print(scan_file_path)
    #problemd = os.path.join('/project/project_462000451/gene/', name_path)
    sufxs = gp.get_all_suffixes(scan_file_path)
    #print(sufxs)
    for sufx in sufxs:
        #print(sufx)
        try:
            out = gp.get_model_outputs(scan_file_path, sufx)
            inp = gp.get_model_inputs(scan_file_path, sufx)
            if len(sufxs) > 4:
                outputs.append(out)
                inputs.append(inp)
                
        #print(inputs)
        #print(outputs)
        except:
            print("Run unsuccessfull for", name_path)
            break

/scratch/project_462000451/daniel/sprint_out/gene/19_2_SSG/1.0-1.8-3.0-4.4-0.05-0.1-0.5-2.8/1.4-3.7-0.07500000000000001-0.07500000000000001-0.5000000000000003/scanfiles0003
Run unsuccessfull for 1.4-3.7-0.07500000000000001-0.07500000000000001-0.5000000000000003
/scratch/project_462000451/daniel/sprint_out/gene/19_2_SSG/1.0-1.8-3.0-4.4-0.05-0.1-0.5-2.8/1.4-4.4-0.07500000000000001-0.07500000000000001-1.65/scanfiles0003
ky =  0.047635959
READING FLUXES
GETTING MODEL INPUTS
NORM GRAD [10.075725, 6.7720666, 6.7720666]
GEOM COEFF {'ggxx': array([1.74198885, 1.56165268, 1.2869666 , 0.99954994, 0.7383124 ,
       0.51515649, 0.33123466, 0.23820219, 0.30986581, 0.58884519,
       1.01066351, 1.41829894, 1.7333436 , 1.93043628, 2.13110002,
       2.29693772, 2.36524111, 2.53021553, 2.51462743, 2.47108221,
       2.34237563, 2.14180018, 1.90331273, 1.6648713 , 1.44385841,
       1.24517211, 1.06129552, 0.90802518, 0.78752162, 0.72569064,
       0.7393616 , 0.84115192, 1.03677125, 1.3026835 , 1.56

In [6]:
df = pd.DataFrame(outputs)# + pd.DataFrame(outputs)
df.head(5)
df = pd.DataFrame(inputs)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,574,575,576,577,578,579,580,581,582,583
0,11.359493,6.911524,6.911524,4.716079,0.379980,0.9025,0.001171,0.042446,1.779091,1.588179,...,1.002287,0.948129,0.863513,0.758164,0.641663,0.521056,0.402354,0.289445,0.171373,0.038769
1,35.505835,17.772834,17.772834,4.755758,0.810369,0.9409,0.000592,0.062852,1.738709,1.552954,...,1.019624,0.951090,0.852881,0.735977,0.613713,0.490413,0.375225,0.268357,0.155956,0.029646
2,69.140712,18.006314,18.006314,4.956822,4.193954,0.9801,0.000141,0.024447,1.800695,1.549510,...,1.035924,0.944902,0.833489,0.706989,0.582593,0.464173,0.351265,0.243234,0.132834,0.007135
3,11.359493,6.911524,6.911524,4.716079,0.379980,0.9025,0.001171,0.042446,1.779091,1.588179,...,1.002287,0.948129,0.863513,0.758164,0.641663,0.521056,0.402354,0.289445,0.171373,0.038769
4,35.505835,17.772834,17.772834,4.755758,0.810369,0.9409,0.000592,0.062852,1.738709,1.552954,...,1.019624,0.951090,0.852881,0.735977,0.613713,0.490413,0.375225,0.268357,0.155956,0.029646


# Train model with the inputs and outputs

In [8]:
inputs_train = np.array(inputs[0:30])
outputs_train = np.array(outputs[0:30])
print(inputs_train.shape)
print(outputs_train.shape)


(30, 584)
(30, 4)


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Dense(128, input_dim=584, activation='relu'),  # Input layer with 584 input features and ReLU activation
    Dense(64, activation='relu'),  
    Dense(4, activation='linear') 
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='mse', 
              metrics=['mae'])

# Train the model
history = model.fit(inputs_train, outputs_train,
                    epochs=100,  
                    batch_size=10,  
                    verbose=1) 

# To access the training loss values and metrics:
print(history.history)

ModuleNotFoundError: No module named 'tensorflow'